In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install langchain langchain-community  -q
!pip install langchain-groq wikipedia groq -q

import os
import wikipedia
import re
from pydantic import BaseModel, Field
from typing import Optional
from kaggle_secrets import UserSecretsClient
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

# API key setup
user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_api_key

# LangChain LLM setup
llm = ChatGroq(model_name="llama3-70b-8192")

# Define how the LLM should summarize Wikipedia content
prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Given the following Wikipedia article content, extract:
- Founder
- Year Founded
- Number of Branches/Campuses
- Number of Employees
- A 4-line summary

Content:
{context}
"""
)

# Combine Wikipedia document with the prompt
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Institution Wikipedia lookup
institution_name = "T John Institute of Technology"
try:
    wiki_content = wikipedia.page(institution_name).content
    doc = Document(page_content=wiki_content)
    result = chain.invoke({"context": [doc]})
    print(result.strip() if isinstance(result, str) else result)
except wikipedia.exceptions.PageError:
    print(f"No page found for '{institution_name}'.")
except wikipedia.exceptions.DisambiguationError as e:
    print(f"Ambiguous institution name. Suggestions: {e.options[:5]}")


Here are the extracted information and summary:

**Founder:** Thomas P John
**Year Founded:** 2006
**Number of Branches/Campuses:** 1 (part of T John Group of Institutions)
**Number of Employees:** Not mentioned

**4-line Summary:**
T. John Institute of Technology (TJIT) is a private engineering college located in Bangalore, Karnataka, India. It was founded in 2006 and is part of T John Group of Institutions. The institute offers undergraduate and postgraduate programs in six streams of engineering and is affiliated to Visvesvaraya Technological University (VTU). TJIT has expanded its facilities and departments over the years, including a digital library, auditorium, and hostel facilities.
